# A3C for Kung Fu

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


### Importing the libraries

In [2]:
import cv2
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.multiprocessing as mp
import torch.distributions as distributions
from torch.distributions import Categorical
import gymnasium as gym
from gymnasium import ObservationWrapper
from gymnasium.spaces import Box

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [3]:
class Network(nn.Module):
  def __init__(self, action_size):
    super().__init__()
    # convolutional layer
    self.conv1 = torch.nn.Conv2d(in_channels =  4, out_channels = 32, kernel_size = (3,3), stride = 2)
    self.conv2 = torch.nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = (3,3), stride = 2)
    self.conv3 = torch.nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = (3,3), stride = 2)
    # flatten layer
    self.flatten = torch.nn.Flatten()
    # 512 = input features, 128 = output features
    self.fc1 = torch.nn.Linear(512, 128)
    # action values, Q values for each action, the expected return
    self.fc2a = torch.nn.Linear(128, action_size)
    # a prediction of the expected return of the current state
    self.fc2s = torch.nn.Linear(128, 1)

  # state = input frames
  def forward(self, state):
    x = self.conv1(state)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.conv3(x)
    x = F.relu(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = F.relu(x)
    action_values = self.fc2a(x)
    # to get the value of the state and not a vector [0] needs to be added in the end
    state_value = self.fc2s(x)[0]
    return action_values, state_value

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Part 2 - Training the AI

### Setting up the environment

In [4]:
class PreprocessAtari(ObservationWrapper):

  def __init__(self, env, height = 42, width = 42, crop = lambda img: img, dim_order = 'pytorch', color = False, n_frames = 4):
    super(PreprocessAtari, self).__init__(env)
    self.img_size = (height, width)
    self.crop = crop
    self.dim_order = dim_order
    self.color = color
    self.frame_stack = n_frames
    n_channels = 3 * n_frames if color else n_frames
    obs_shape = {'tensorflow': (height, width, n_channels), 'pytorch': (n_channels, height, width)}[dim_order]
    self.observation_space = Box(0.0, 1.0, obs_shape)
    self.frames = np.zeros(obs_shape, dtype = np.float32)

  def reset(self):
    self.frames = np.zeros_like(self.frames)
    obs, info = self.env.reset()
    self.update_buffer(obs)
    return self.frames, info

  def observation(self, img):
    img = self.crop(img)
    img = cv2.resize(img, self.img_size)
    if not self.color:
      if len(img.shape) == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.astype('float32') / 255.
    if self.color:
      self.frames = np.roll(self.frames, shift = -3, axis = 0)
    else:
      self.frames = np.roll(self.frames, shift = -1, axis = 0)
    if self.color:
      self.frames[-3:] = img
    else:
      self.frames[-1] = img
    return self.frames

  def update_buffer(self, obs):
    self.frames = self.observation(obs)

def make_env():
  env = gym.make("KungFuMasterDeterministic-v4", render_mode = 'rgb_array')
  env = PreprocessAtari(env, height = 42, width = 42, crop = lambda img: img, dim_order = 'pytorch', color = False, n_frames = 4)
  return env

env = make_env()
# the state is a stack of 4 greyscale images of dimension 42 by 42
state_shape = env.observation_space.shape
number_actions = env.action_space.n
print("Observation shape:", state_shape)
print("Number actions:", number_actions)
print("Action names:", env.env.env.get_action_meanings())

Observation shape: (4, 42, 42)
Number actions: 14
Action names: ['NOOP', 'UP', 'RIGHT', 'LEFT', 'DOWN', 'DOWNRIGHT', 'DOWNLEFT', 'RIGHTFIRE', 'LEFTFIRE', 'DOWNFIRE', 'UPRIGHTFIRE', 'UPLEFTFIRE', 'DOWNRIGHTFIRE', 'DOWNLEFTFIRE']


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_action_meanings to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_action_meanings` for environment variables or `env.get_wrapper_attr('get_action_meanings')` that will search the reminding wrappers.
  logger.warn(


### Initializing the hyperparameters

In [5]:
learning_rate = 1e-4
discount_factor = 0.99
# this variable is specific to A3C models because this model trains multiple agents in multiple environments in parallel
number_environments = 10

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Implementing the A3C class

In [6]:
class Agent():

  def __init__(self, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # initialize the action size
    self.action_size = action_size
    # initialize the brain
    self.network = Network(action_size).to(self.device)
    # initialize the optimizer
    # inside the Adam optimizer (self.network.parameters() = the models parameters to optimize)
    # the learning rate is one of the essential parameters of the optimizer
    self.optimizer = torch.optim.Adam(self.network.parameters(), lr = learning_rate)

  # this method allows the agent to play an action based on the softmax strategy
  def act(self, state):
    # check if state's dimension is 3 (not inside a batch)
    if state.ndim == 3:
      state = [state]   # add extra dimension to make it 4 including the batch
    # convert the state into a torch tensor
    state = torch.tensor(state, dtype = torch.float32, device = self.device)
    # get the action values and disregard the current state value
    action_values, _ = self.network.forward(state)
    # dim is the dimension along which softmax will be computed
    # dim = -1 : it means that the soft max should be applied across the last dimension, which is the dimension representing the different actions
    policy = F.softmax(action_values, dim = -1)   # softmax gives us a distribution of probabilities
    return np.array([np.random.choice(len(p), p = p) for p in policy.detach().cpu().numpy()])  # policy starts as a tensor, then it is detached from the GPU and it is sent to cpu, and then it is converted in a numpy array

  # Update A3C parameters (weights of nn) in each step of the training
  # The method will take as input a batch of states, a batch of actions, a batch of rewards, a batch of next_states.
  # To simplify the notations of the code a single state, action, reward, next_state, and done is used, to avoid too many S's in the code.

  def step(self, state, action, reward, next_state, done):
    batch_size = state.shape[0] # state is a batch. By using the shape attribute and 0 index we get the number of state observations in the batch
    # convert state, reward, next_state, and done into torch tensors; action will be converted later on
    state = torch.tensor(state, dtype = torch.float32, device = self.device)
    next_state = torch.tensor(next_state, dtype = torch.float32, device = self.device)
    reward = torch.tensor(reward, dtype = torch.float32, device = self.device)
    done = torch.tensor(done, dtype = torch.bool, device = self.device).to(dtype = torch.float32)
    # action values and state value
    action_values, state_value = self.network(state)
    # the next action_values are disregarded, and the model is calculatin only the next_state_value
    _, next_state_value = self.network(next_state)
    # compute target state value by using the Bellman equation
    target_state_value = reward + discount_factor * next_state_value * (1-done)
    # implementation of A3C model
    # advantage
    advantage = target_state_value - state_value
    # computation of actor's loss
    probs = F.softmax(action_values, dim = -1)    # probability distribution of the action values
    logprobs = F.log_softmax(action_values, dim = -1)   # log probability distribution of the action values
    # entropy
    entropy = -torch.sum(probs * logprobs, axis = -1)
    # this array will be used to select the log probabilites of the actions that are taken in the batch
    batch_idx = np.arange(batch_size)  # this array length same as the length of the batch_size

    # logp_actions will be a tensor containing the log probabilities of the actions that were actually taken in the batch of states
    # batch_idx is used to select the row index for each sample in the batch.
    # action is used to select the column index for each sample in the batch.
    logp_actions = logprobs[batch_idx, action]

    # actor's loss
    actor_loss = -(logp_actions * advantage.detach()).mean() - 0.001 * entropy.mean()  # (0.001 * mean(entropy)) is an exploration feature which will balance the importance of this entropy
    # critic loss
    critic_loss = F.mse_loss(target_state_value.detach(), state_value) # detach() prevents the gradients from flowing into the target computations
    # total loss
    total_loss = actor_loss + critic_loss

    # reset the optimizer
    self.optimizer.zero_grad()
    # backpropagation
    total_loss.backward()
    # update neural network's weight with the optimizer
    self.optimizer.step()


### Initializing the A3C agent

In [7]:
agent = Agent(number_actions)

### Evaluating our A3C agent on a certain number of episodes


In [8]:
def evaluate(agent, env, n_episodes = 1 ):    # n_episodes = number of episodes the environment will be evaluated
  episodes_rewards = []
  for _ in range(n_episodes):     # loop variable not used, hence disregarded with _
    state, _ = env.reset()
    total_reward = 0
    while True:
      action = agent.act(state)
      state, reward, done, info, _ = env.step(action[0])
      total_reward += reward
      if done:
        break
    episodes_rewards.append(total_reward)
  return episodes_rewards

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Testing multiple agents on multiple environments at the same time

In [9]:
# asynchronous feature will allow multiple environments to be created simultaneously

class EnvBatch:
  def __init__(self, n_envs = 10):
    self.envs = [make_env() for _ in range(n_envs)]

  # reset the multiple environments at the same time
  def reset(self):
    _states = []
    for env in self.envs:
      _states.append(env.reset()[0])
    return np.array(_states)

  # step into multiple environments simultaneously
  def step(self, actions):
    next_states, rewards, dones, infos, _ = map(np.array, zip(*[env.step(a) for env, a in zip(self.envs, actions)]))
    for i in range(len(self.envs)):
      if dones[i]:
        next_states[i] = self.envs[i].reset()[0]
    return next_states, rewards, dones, infos


### Training the A3C agent

In [10]:
import tqdm

env_batch = EnvBatch(number_environments)
batch_states = env_batch.reset()

with tqdm.trange(0, 3001) as progress_bar:
  for i in progress_bar:
    batch_actions = agent.act(batch_states)
    batch_next_states, batch_rewards, batch_dones, _ = env_batch.step(batch_actions)
    batch_rewards *= 0.01
    agent.step(batch_states, batch_actions, batch_rewards, batch_next_states, batch_dones)
    batch_states = batch_next_states
    if i % 1000 == 0:
      print("Average agent reward: ", np.mean(evaluate(agent, env, n_episodes = 10)))

  0%|          | 0/3001 [00:00<?, ?it/s]<ipython-input-6-177b48999c80>:64: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  critic_loss = F.mse_loss(target_state_value.detach(), state_value) # detach() prevents the gradients from flowing into the target computations
<ipython-input-6-177b48999c80>:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  state = torch.tensor(state, dtype = torch.float32, device = self.device)
  0%|          | 4/3001 [00:38<6:06:55,  7.35s/it] 

Average agent reward:  520.0


 34%|███▎      | 1006/3001 [01:42<1:01:06,  1.84s/it]

Average agent reward:  760.0


 67%|██████▋   | 2006/3001 [02:45<25:35,  1.54s/it]

Average agent reward:  750.0


100%|██████████| 3001/3001 [03:47<00:00, 13.21it/s]

Average agent reward:  930.0


## Part 3 - Visualizing the results

In [11]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env):
  state, _ = env.reset()
  done = False
  frames = []
  while not done:
    frame = env.render()
    frames.append(frame)
    action = agent.act(state)
    state, reward, done, _, _ = env.step(action[0])
  env.close()
  imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, env)

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
